# Dynamics of the clonal competition model
To generate the data: 

`cargo r --release -- --mean-std 0.20 0.05 -r 1 -c 10000 --snapshots=5,10,15,20,20,25,30,35,40,45,50,55,60,65,70,75,80,80,85,90,95,100 --subsamples=10000,10000,10000,400,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,10000,400,10000,10000,10000,10000 -y 101 --sequential test exp-moran`

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pathlib import Path
from futils import snapshot
from hscpy import realisation, variant
from hscpy.figures import sfs as sfs_fig
from hscpy.figures import ToCellFrequency

from matplotlib import font_manager

font_dirs = [
    "../.local/share/fonts/otf/TexGyreHeros/"
]  # The path to the custom font file.
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

FIGSIZE = (3, 2)

In [ ]:
burden_ = realisation.load_realisations(
    Path("../hsc/test/10000cells/burden/"),
    realisation.RealisationKind.BURDEN,
    filtering=False,
)


# "#b3b3b3"
colors = {
    20: "#66c2a5",
    30: "#8da0cb",
    40: "#e78ac3",
    50: "#a6d854",
    60: "#ffd92f",
    70: "#e5c494",
    80: "#fc8d62",
}
fig, ax = plt.subplots(1, 1, figsize=FIGSIZE, layout="constrained")
for age, s in burden_.items():
    c = colors.get(age)
    if c:
        tot = sum(s[0].burden.values())
        ax.bar(
            s[0].burden.keys(),
            [ele / tot for ele in s[0].burden.values()],
            width=1,
            # label=f"{int(age)} y.o.",
            color=c,
        )
# ax.set_ylim([10 ** (-6), 1])
# ax.legend(frameon=True)
ax.set_ylabel("Density")
ax.set_xlabel("Single-cell burden")
plt.savefig("burden_model_full_pop.svg")
plt.show()

In [ ]:
sfs_ = realisation.load_realisations(
    Path("../hsc/test/10000cells/sfs/"),
    realisation.RealisationKind.SFS,
    filtering=False,
)
colors = {20: "#66c2a5", 80: "#fc8d62"}
fig, ax = plt.subplots(1, 1, figsize=FIGSIZE, layout="constrained")
for age, s in sfs_.items():
    if age in colors.keys():
        sfs_fig.plot_sfs(
            ax,
            s[0].sfs,
            normalise=True,
            normalise_x=ToCellFrequency(s[0].parameters.sample),
            options=None,
            ls="",
            marker=".",
            label=f"{int(age)} y.o.",
            c=colors[age],
        )
ax.set_ylim([10 ** (-6), 1])
ax.set_xlim([10 ** (-4), 1])
ax.set_yscale("log")
ax.set_xscale("log")
# expectations
x_ = np.arange(0.01, 10_000, 5)
ax.plot(x_ / 10_000, 1 / x_, ls="--", c="black", label="1/f", alpha=0.8)
ax.plot(x_ / 10_000, 1 / x_**2, ls="-", c="black", label="1/f^2", alpha=0.8)
ax.legend(frameon=True, handletextpad=-0.2)
plt.savefig("sfs_model_full_pop.svg")
plt.show()

In [ ]:
one_over_f_sq = realisation.compute_variants(
    realisation.Correction.ONE_OVER_F_SQUARED, 10_000, 400
)
# theoretical homeostatic neutral SFS data, from Nate's paper in Elife: for each patient (skipping the neonates)
# I evolved until their specific age, and then sampled to the same size as in the data
one_over_f_csv = "predictions_1_over_f/homeostasisSFS_pid3.csv"

In [ ]:
sfs_ = realisation.load_realisations(
    Path("../hsc/test/400cells/sfs/"), realisation.RealisationKind.SFS, filtering=False
)

fig, ax = plt.subplots(1, 1, figsize=FIGSIZE, layout="constrained")
for (age, s), c in zip(sfs_.items(), ("#66c2a5", "#fc8d62")):
    sfs_fig.plot_sfs(
        ax,
        s[0].sfs,
        normalise=True,
        normalise_x=ToCellFrequency(s[0].parameters.sample),
        options=None,
        ls="",
        marker=".",
        label=f"{int(age)} y.o.",
        c=c,
    )
sfs_fig.plot_sfs_correction(
    ax,
    one_over_f_sq,
    normalise=True,
    options=None,
    normalise_x=ToCellFrequency(s[0].parameters.sample),
    ls="-",
    alpha=0.8,
    c="black",
    label="1/f^2",
)
# add Nate's predictions
normalisation_x = ToCellFrequency(s[0].parameters.sample)
one_over_f = pd.read_csv(one_over_f_csv)
one_over_f.drop(index=one_over_f[one_over_f["_f"] == 0.0].index, inplace=True)
sfs_one_over_f = {
    cell: muts
    for cell, muts in zip(
        (one_over_f["_f"] * normalisation_x.nb_cells).tolist(),
        one_over_f["n_f"].tolist(),
    )
}
sfs_fig.plot_sfs(
    ax,
    snapshot.Histogram(sfs_one_over_f),
    normalise=True,
    normalise_x=normalisation_x,
    options=None,
    color="black",
    alpha=0.8,
    linestyle="--",
    label="1/f",
)
ax.set_ylim([10 ** (-6), 1])
ax.set_xlim([10 ** (-3), 1])
ax.legend(frameon=True, handletextpad=-0.2)
plt.savefig("sfs_model.svg")
plt.show()

In [ ]:
with open("../hsc/test/rates/260.csv", "r") as f:
    rates = [float(ele) for ele in f.readlines()[0].split(",") if ele][
        1:
    ]  # rm wild type
rates

In [ ]:
def pick_c(rate: float) -> str:
    if rate <= 1.15:
        return "#377eb8"
    if rate <= 1.20:
        return "#4daf4a"
    if rate <= 1.25:
        return "#984ea3"
    return "#e41a1c"

In [ ]:
var_fracs = variant.load_all_var_frac_by_age(
    Path("../hsc/test/10000cells/variant_fraction"),
)
var_fracs = dict(sorted(var_fracs.items()))
v = np.array([v[0].variant_fractions for v in var_fracs.values()]).T
t = [v for v in var_fracs.keys()]
t = np.tile(np.array(t), (v.shape[0], 1))

fig, ax = plt.subplots(1, 1, figsize=(6, 2), layout="constrained")
for i in range(0, v.shape[0]):
    # if (v[i] >= 0.01).any():
    ax.plot(t[i], v[i], ls="-", c=pick_c(rates[i]), alpha=0.9, lw=1.5)
ax.fill_betweenx(y=[0, 10 ** (-2)], x1=0, x2=100, color="black", alpha=0.1)
ax.set_ylabel("Clone sizes")
ax.set_xlabel("Age (years)")
ax.set_ylim([0.001, 1])
ax.set_xlim([0, 100])
ax.set_yscale("log")
plt.savefig("clone_sizes_model.svg")
plt.show()

In [ ]:
colors = {
    20: "#66c2a5",
    30: "#8da0cb",
    40: "#e78ac3",
    50: "#a6d854",
    60: "#ffd92f",
    70: "#e5c494",
    80: "#fc8d62",
}
fig, ax = plt.subplots(1, 1, figsize=FIGSIZE, layout="constrained")
for age, c in colors.items():
    idx = int(age / 5)
    y = np.where(v[:, idx] >= 0.01, v[:, idx], 0).sum() * 100
    ax.bar(age, y, width=5, color=c)
    ax.text(
        s=f"{(v[:, idx] >= 0.01).sum()}",
        x=age - 1.5,
        y=y + 5 if y < 90 else 100,
        fontsize="small",
    )
ax.set_ylabel("Clonal fraction [%]")
ax.set_xlabel("Age (years)")
ax.set_ylim([0, 110])
ax.set_xlim([15, 85])
plt.savefig("clone_sizes_barplot_model.svg")
plt.show()